In [2]:
from time import time
import polars as pl
import pandas as pd
import numpy as np
import pyarrow

import matplotlib.pyplot as plt
import seaborn as sns

df = pl.scan_csv(r"../datasets/2019-Nov.csv")

In [3]:
df.head().collect(streaming=True)

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
str,str,i64,i64,str,str,f64,i64,str
"""2019-11-01 00:…","""view""",1003461,2053013555631882655,"""electronics.sm…","""xiaomi""",489.07,520088904,"""4d3b30da-a5e4-…"
"""2019-11-01 00:…","""view""",5000088,2053013566100866035,"""appliances.sew…","""janome""",293.65,530496790,"""8e5f4f83-366c-…"
"""2019-11-01 00:…","""view""",17302664,2053013553853497655,null,"""creed""",28.31,561587266,"""755422e7-9040-…"
"""2019-11-01 00:…","""view""",3601530,2053013563810775923,"""appliances.kit…","""lg""",712.87,518085591,"""3bfb58cd-7892-…"
"""2019-11-01 00:…","""view""",1004775,2053013555631882655,"""electronics.sm…","""xiaomi""",183.27,558856683,"""313628f1-68b8-…"


In [23]:
%%time

# Counting all the element of the Lazy Dataframe

df.select(pl.count()).collect().item()

CPU times: total: 6.72 s
Wall time: 6.71 s


67501979

In [27]:
%%time

# Counting the unique numbers of products into the DataFrame

df.select([
    pl.col('product_id').n_unique()
]).collect(streaming=True)

CPU times: total: 13 s
Wall time: 7.18 s


product_id
u32
190662


In [30]:
%%time

# With groupby we can group elements by columns' values and perform some aggregation operations with lazy dataframe

df.groupby("brand").agg(
    pl.count().alias("num_items")
).collect(streaming=True).head()

CPU times: total: 3.44 s
Wall time: 5.94 s


brand,num_items
str,u32
"""pioneer""",201799
"""moony""",1183
"""kipocket""",1376
"""niio""",1136
"""master""",1864


In [5]:
%%time

# Aggregation with sorting

df.groupby("brand").agg([
    pl.count().alias("num_items_per_brand")
]).sort("num_items_per_brand", descending=True).collect(streaming=True).head()

CPU times: total: 6.36 s
Wall time: 6.85 s


brand,num_items_per_brand
str,u32
null,9218235
"""samsung""",7889245
"""apple""",6259379
"""xiaomi""",4638062
"""huawei""",1410126


In [1]:
%%time

#Groupby on two columns and sorting
df_brand_event = df.groupby(["brand", "event_type"]).agg(pl.count()).sort(["count"], descending=True)

df_brand_event.collect(streaming=True).head()

NameError: name 'df' is not defined